In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
from deeprte.config import make_config
from deeprte.model.tf.input_pipeline import shard
config = make_config()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config

best_model_eval_metric: ''
best_model_eval_metric_higher_is_better: true
checkpoint_interval_type: null
data:
  buffer_size: 5000
  evaluation:
    batch_size: 4
  is_split_datasets: false
  max_intra_op_parallelism: 1
  seed: 42
  threadpool_size: 48
  training:
    batch_size: 8
    collocation_sizes: 500
    num_epochs: 100
    num_train_examples: 6
    repeat: 1
eval_initial_weights: false
experiment_kwargs: {}
interval_type: secs
log_all_train_data: false
log_tensors_interval: 60
log_train_data_interval: 120.0
logging_interval_type: null
max_checkpoints_to_keep: 5
one_off_evaluate: false
random_mode_eval: same_host_same_device
random_mode_train: unique_host_unique_device
random_seed: 42
save_checkpoint_interval: 300
train_checkpoint_all_hosts: false
training_steps: 10000

In [4]:
shard(config.data,0,2,True)

(<tf.Tensor: shape=(), dtype=int32, numpy=4>,
 <tf.Tensor: shape=(), dtype=int32, numpy=7>)

In [5]:
data_path = "/workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_shape.mat"

from deeprte.data.pipeline import DataPipeline
data_pipeline = DataPipeline()
data = data_pipeline.process(data_path)

from deeprte.model.tf.rte_dataset import np_to_tensor_dict

tf_data = np_to_tensor_dict(data)

from deeprte.model.tf.rte_dataset import divide_batch_feat
batched_data, unbatched_data = divide_batch_feat(tf_data)

In [6]:
import tensorflow as tf

tf.nest.map_structure(lambda x:x.shape, tf_data)

{'sigma': TensorShape([10, 1681, 2]),
 'psi_label': TensorShape([10, 40344]),
 'boundary': TensorShape([10, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'scattering_kernel': TensorShape([10, 40344, 24]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [7]:
from deeprte.model.tf.input_pipeline import load_and_split_data,process_features,np_example_to_features

ds, unbatched_feat = load_and_split_data(tf_data, config.data, True)

In [8]:
tfds_feat = process_features(ds, unbatched_feat, is_training = True, batch_sizes=[2,3], collocation_sizes=500)

In [9]:
tf.nest.map_structure(lambda x:x.shape, next(tfds_feat))

{'sigma': (2, 3, 1681, 2),
 'psi_label': (2, 3, 500),
 'boundary': (2, 3, 1968),
 'scattering_kernel': (2, 3, 500, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 500, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [10]:
data_config = config.data

In [13]:
tfds_input = np_example_to_features(data, batch_sizes=[2,5], data_config = data_config)

In [14]:
tf.nest.map_structure(lambda x:x.shape, next(tfds_input))

{'sigma': (2, 5, 1681, 2),
 'psi_label': (2, 5, 40344),
 'boundary': (2, 5, 1968),
 'scattering_kernel': (2, 5, 40344, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 40344, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [15]:
data_config.is_split_datasets = True

In [22]:
tfds_train_input, tfds_eval_input = np_example_to_features(data, batch_sizes=[2,2], data_config = data_config)

In [23]:
tf.nest.map_structure(lambda x:x.shape, next(tfds_eval_input))

{'sigma': (2, 2, 1681, 2),
 'psi_label': (2, 2, 40344),
 'boundary': (2, 2, 1968),
 'scattering_kernel': (2, 2, 40344, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 40344, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [24]:
tf.nest.map_structure(lambda x:x.shape, next(tfds_train_input))

{'sigma': (2, 2, 1681, 2),
 'psi_label': (2, 2, 500),
 'boundary': (2, 2, 1968),
 'scattering_kernel': (2, 2, 500, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 500, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}